In [ ]:
# 图片预处理, 
# 透视变换, 视角拉正.
# tesseract进行识别.

In [12]:
def cv_show(name, image):
    cv2.imshow(name, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 对图片进行同一的resize
# 封装resize功能.
def resize(image, width=None, height=None, inter=cv2.INTER_AREA):
    dim = None
    (h, w) = image.shape[:2]
    if width is None and height is None:
        return image
    # 指定了resize的height
    if width is None:
        r = height / float(h)
        dim = (int(w * r), height)
    # 指定了width
    else:
        r = width / float(w)
        dim = (width, int(h * r))
    resized = cv2.resize(image, dim, interpolation=inter)
    return resized

In [31]:
import numpy as np
import cv2 

# 读取图片
image = cv2.imread('./images/receipt.jpg')
# 计算比例. 限定高度500
ratio = image.shape[0] / 500.0
orig = image.copy()

# 对图片进行resize
image = resize(orig, height=500)
cv2.imwrite('./resize1.jpg', image)
cv_show('edged', image)

In [32]:
# 图片预处理
# 灰度化处理
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
# 高斯平滑
gray = cv2.GaussianBlur(gray, (5, 5), 0)
# 边缘检测
edged = cv2.Canny(gray, 75, 200)
cv_show('edged', edged)

In [20]:
# 轮廓检测
cnts = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)[0]
# 按照面积排序
cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
image_contours = cv2.drawContours(image.copy(), cnts[0], -1, (0, 0, 255), 2)
cv_show('image_contours', image_contours)

In [27]:
# 遍历轮廓找出最大的轮廓. 
for c in cnts:
    # 计算轮廓周长
    perimeter = cv2.arcLength(c, True)
    # 多边形逼近.
    approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
    
    if len(approx) == 4:
        screen_cnt = approx
        break

image_contours = cv2.drawContours(image.copy(), [screen_cnt], -1, (0, 0, 255), 2)
cv_show('image_contours', image_contours)

In [22]:
# 进行透视变换.
def order_points(pts):
    # 创建全是0的矩阵, 来接收等下找出来的4个角的坐标.
    rect = np.zeros((4, 2), dtype='float32')
    s = pts.sum(axis=1)
    # 左上的坐标一定是x,y加起来最小的坐标. 右下的坐标一定是x,y加起来最大的坐标.
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    
    # 右上角的x,y相减的差值一定是最小的. 
    # 左下角的x,y相减的差值, 一定是最大.
    diff = np.diff(pts, axis=1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

# 把透视变换功能封装成一个函数
def four_point_transform(image, pts):
    # 对输入的4个坐标排序
    rect = order_points(pts)
    (tl, tr, br, bl) = rect
    
    # 空间中两点的距离
    widthA = np.sqrt((br[0] - bl[0]) ** 2 + (br[1] - bl[1]) ** 2)
    widthB = np.sqrt((tr[0] - tl[0]) ** 2 + (tr[1] - tl[1]) ** 2)
    max_width = max(int(widthA), int(widthB))
    
    heightA = np.sqrt((tr[0] - br[0]) ** 2 + (tr[1] - br[1]) ** 2)
    heightB = np.sqrt((tl[0] - bl[0]) ** 2 + (tl[1] - bl[1]) ** 2)
    max_height = max(int(heightA), int(heightB))
    
    # 构造变换之后的对应坐标位置.
    dst = np.array([
        [0, 0],
        [max_width - 1, 0],
        [max_width - 1, max_height - 1],
        [0, max_height - 1]], dtype='float32')
    
    # 计算变换矩阵
    M = cv2.getPerspectiveTransform(rect, dst)
    # 透视变换
    warped = cv2.warpPerspective(image, M, (max_width, max_height))
    return warped

In [23]:
warped = four_point_transform(orig, screen_cnt.reshape(4, 2) * ratio)
# 二值处理
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
ref = cv2.threshold(warped, 100, 255, cv2.THRESH_BINARY)[1]
cv_show('ref', ref)

In [24]:
# 把处理好的图片写入图片文件.
cv2.imwrite('./scan.jpg', ref)

True

In [26]:
import pytesseract
from PIL import Image

# pytesseract要求的image不是opencv读进来的image, 而是pillow这个包, 即PIL,按照 pip install pillow
text = pytesseract.image_to_string(Image.open('./scan.jpg'))
print(text)

fp ACK PSSING AND STANCE ETE ALING FPEXELTS

On Line rq we manipulate the top left pixel in the im-
age, which is located at coordinate 10,01 and set if to have
avalue of (0, 0, 255). If we were reading. this pixel value
in RGB format, we would have a value of % for red, 0 tor
green, and 255 tor blue, thus making ita pure blue color

However, as | mentioned above, we need to take special
care when working with OpenCY. Our pixels are actualls
stored in BGR format, not RGB format.

We actually read this pixel as 255 for red, 0 for green, and
0 for blue, making it a red color, not a blue color.

After setting the top-left pixel to have a red color on Line
14, we then grab the pixel value and print it back to con-
sole on Lines 15 and 16, just to demonstrate that we have
indeed successfully changed the color of the pixel.

Accessing and setting a single pixel value is simple enough,
but what if we wanted to use NumPv’s array slicing capa-
bilities to access larger rectangular portions of t